In [1]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [11]:
# Data of Player Stats for Top 500 Players (player name, their country, what hand they play)
player_data = read_delim("data/player_stats.csv", delim = ",") %>%
            filter(Country == 'Canada' | Country == 'United States') %>%
            select(Name, Plays) %>%
            mutate(Plays = as.factor(Plays))
player_data

# Data of Game results for Top 500 Players from 2017-2019 (court surface, winner, winner serving hand)
game_data = read_delim("data/atp2017-2019.csv", delim = ",") %>%
            select(surface, winner_name, winner_hand)
game_data

New names:
* `` -> ...1

Rows: 500 Columns: 38

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



Name,Plays
<chr>,<fct>
Sekou Bangoura,Right-handed
Felix Auger Aliassime,Right-handed
Steven Diez,Right-handed
Taylor Harry Fritz,Right-handed
Sam Querrey,Right-handed
Ryan Harrison,Right-handed
Filip Peliwo,Right-handed
Peter Polansky,Right-handed
Mitchell Krueger,Right-handed


New names:
* `` -> ...1

Rows: 6866 Columns: 50

── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (16): tourney_id, tourney_name, surface, tourney_level, winner_seed, win...
dbl (34): ...1, draw_size, tourney_date, match_num, winner_id, winner_ht, wi...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



surface,winner_name,winner_hand
<chr>,<chr>,<chr>
Hard,Kei Nishikori,R
Hard,Daniil Medvedev,R
Hard,Kei Nishikori,R
Hard,Jo Wilfried Tsonga,R
Hard,Daniil Medvedev,R
Hard,Jeremy Chardy,R
Hard,Kei Nishikori,R
Hard,Jo Wilfried Tsonga,R
Hard,Alex De Minaur,R
